# Run apAdm
Goal:
Run qp-Admix for identifying question about Sardinian Demography.
Use a wrapper to run it with various parameters, for various populations.

Use two different set of outgroups: m13 and anc_euro. Save them in different subfolders.

In [10]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time

# For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
# Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-e-16-233.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 28
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


# Define Some Standard Outgroups

In [5]:
### Parameters for qpAdm_run are set there!
a12 = ["Mota", "Ust_Ishim", "Kostenki14", "GoyetQ116-1", "Vestonice16", "MA1",
           "ElMiron", "Villabruna", "EHG", "CHG", "Natufian",
           "Levant_N"] # List of ancient European populations for right populations
distal = ["Anatolia_N", "WHG", "Steppe_EMBA", 
          "Iran_N", "Morocco_EN"]
print("# Anc_Euro: %i populations" % len(a12))
print("# Distal sources: %i populations" % len(distal))

# Anc_Euro: 12 populations
# Distal sources: 5 populations


In [6]:
def qpAdm_run(leftpops, rightpops, output_file, 
              input_folder = "./eigenstrat/combined/", 
              input_file="punic.v44",
              par_file_folder = "./parfiles/", 
              input_ind_suff="_mod1", 
              output_folder ="./output/qpAdm/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=False):
    """Run qpAdm. Write temporary parfile and run the analysis
    leftpops: List of left populations
    rightpops: List of right populations
    input_folder: Where to find the input files
    par_file_folder: Folder of the parameterfile
    input_file: The name of the input file
    input_ind_suff: Suffix of .ind file. To allow modified populations
    input_ind: name of the input .ind file. If given - write over default"""
    
    parfile_path = par_file_folder + "parfile_qpAdm"
    left_path, right_path = par_file_folder + "left", par_file_folder + "right"
    
    ### Create the parfile:
    with open(parfile_path, 'w') as f:
        f.write("%s\n" % ("DIR: " + input_folder))
        f.write("%s\n" % ("S1: " + input_file))
        
        indline = "indivname: DIR/S1" + input_ind_suff + ".ind"
        f.write("%s\n" % indline)
        f.write("%s\n" % "snpname: DIR/S1.snp")
        f.write("%s\n" % "genotypename: DIR/S1.geno")
        f.write("%s\n" % ("popleft: " + left_path))
        f.write("%s\n" % ("popright: " + right_path))
        f.write("%s\n" % "details: YES")   
        if all_snps==True:
            f.write("%s\n" % "allsnps: YES")
    
    ### Write leftpops rightpops:       
    with open(left_path, 'w') as f:
        f.write("\n".join(leftpops))
        
    with open(right_path, 'w') as f:
        f.write("\n".join(rightpops))
      
    ### Run qpAdm
    start = time()
    output_path = output_folder + output_file
    !$path_bin_qpAdm -p $parfile_path > $output_path
    end = time()
    print("Runtime: %2f" % (end - start))
    return 0

############################################
### Identify Individuals to run

def load_iids_from_indfile(path_ind, string, 
                           col="clst", col_iid="iid"):
    """Load IIDs from Ind File
    Return List of IIDs"""
    df_ind = pd.read_csv(path_ind, delim_whitespace=True, header=None)
    df_ind.columns=["iid", "sex", "clst"]
    idx = df_ind[col].str.contains(string)
    ls = df_ind[idx][col_iid].values
    return ls

def remove_individuals(df, remove_list=["_d"], iid_col="iid"):
    """Remove indivdiuals from PCA dataframe"""
    idx = df[iid_col].str.contains("|".join(remove_list))
    df = df[~idx].copy()
    print(f"Filtering to {len(df)}/{len(idx)}")
    return df

def get_sub_pops(df, pop_list=[], pop_col="clst"):
    """Get Sub Populations"""
    idx = df[pop_col].str.contains("|".join(pop_list))
    df = df[idx].copy()
    print(f"Found: {len(df)}/{len(idx)}")
    return df

def get_sub_pops_exact(df, pop_list=[], pop_col="clst"):
    """Get Sub Populations"""
    idx = df[pop_col].isin(pop_list)
    df = df[idx].copy()
    print(f"Found: {len(df)}/{len(idx)}")
    return df

def get_sub_iid(df, pops=[""], iid_col="iid"):
    """Remove indivdiuals from PCA dataframe"""
    idx = df[iid_col].str.contains("|".join(pops))
    df = df[idx].copy()
    print(f"Found: {len(df)}/{len(idx)}")
    return df

### Get Dataframe with .anno clusters, merged with qpAdm eigenstrat IIDs

In [4]:
min_snp = 30000
ind_merged="./eigenstrat/combined/punic.v44.ind"          # What .ind to load
df = pd.read_csv(ind_merged, delim_whitespace=True, header=None)
df.columns=["iid", "sex", "clst"]
df = remove_individuals(df, remove_list=["_d"])
print(f"Loaded {len(df)} Individuals")

df_meta = pd.read_csv("/n/groups/reich/hringbauer/Data/v42-v44/v44.0.anno.csv", sep=",")
df2 = df_meta[["iid", "Master ID", "loc", "n_cov_snp", "mean_cov", "sex"]]
df = pd.merge(df, df2, on="iid", how="left")
idx = (df["n_cov_snp"]<min_snp)
df = df[~idx]
df.loc[df["loc"].isnull(), "loc"]="not assigned"
print(f"Filtered to {len(df)} Individuals based on Min SNP: {min_snp}")

df = df.sort_values(by="n_cov_snp", ascending=False)
dup = (df["Master ID"].duplicated() & ~df["Master ID"].isnull())
df = df[~dup].copy().reset_index(drop=True)
print(f"Filtered to {len(df)} based on duplicates")

Filtering to 1037/1150
Loaded 1037 Individuals
Filtered to 980 Individuals based on Min SNP: 30000
Filtered to 912 based on duplicates


### Diverse Modelling Runs for single Individual

In [5]:
%%time

iid = "Tunisia_N"  # The Sicilian "inbred individual"
sources = ["Morocco_Iberomaurusian", "I12433"]

add_out = []
leftpops = [iid] + sources

qpAdm_run(leftpops = leftpops, 
          rightpops = a12+add_out, 
          output_file = ".".join(leftpops) + ".log", 
          input_folder = "./eigenstrat/combined/", 
          input_file = "punic.v46.3",
          par_file_folder = "./parfiles/", 
          input_ind_suff = "_ind2", 
          output_folder = "./output/qpAdm/diverse/", 
          path_bin_qpAdm = "/n/groups/reich/hringbauer/git/AdmixTools/bin/qpAdm",
          all_snps=True)

^C
Runtime: 2.645691
CPU times: user 61.7 ms, sys: 25.6 ms, total: 87.3 ms
Wall time: 2.69 s


0

# v46.3: Test what additional European Source via Punic Cline
Where does the Punic cline point to?

In [11]:
%%time
target = "PunicCline"  # The Sicilian "inbred individual"
s1 = "PunicAfrican"

sources2 = ["Spain_IA_Tartessian", "Spain_IA_Celt", 
            "Italy_Sardinia_BA_Nuragic",  "Italy_Sicily_IA_Sicani", "Italy_Sicily_IA_Polizzello", 
            "Greece_BA_Mycenaean", "Israel_Phoenician"]

for s2 in sources2:
    leftpops = [target, s1, s2]

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + ["Morocco_Iberomaurusian"], 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic.v46.3",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_punic_clst", 
              output_folder = "./output/qpAdm/v46.3/eu_source/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/git/AdmixTools/bin/qpAdm",
              all_snps=True)

Runtime: 70.454916
CPU times: user 1.68 s, sys: 366 ms, total: 2.05 s
Wall time: 1min 10s


# v46.3: Test what African Source on Punic Cline

In [12]:
%%time
target = "PunicCline"  # The Sicilian "inbred individual"
s1 = "Italy_Sicily_IA_Polizzello"

sources2 = ["PunicAfrican", "Tunisia_N", "Algeria_IA", "Morocco_LN.SG", "Morocco_EN.SG",
           "Spain_BellBeaker_oAfrican", "Italy_Sicily_MBA_oAfrican", 
            "Italy_Sardinia_C_oAfrican_new", "Spain_EBA_Africa_all", "Nigeria_Medieval"] 

for s2 in sources2:
    leftpops = [target, s1, s2]

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + ["Morocco_Iberomaurusian"], 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic.v46.3",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_punic_clst", 
              output_folder = "./output/qpAdm/v46.3/afr_source/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/git/AdmixTools/bin/qpAdm",
              all_snps=True)

Runtime: 70.315276
Runtime: 72.097692
Runtime: 70.810354
Runtime: 70.837760
Runtime: 71.374343
Runtime: 70.586507
Runtime: 74.052516
Runtime: 74.255633
Runtime: 74.569834
Runtime: 73.747529
CPU times: user 16.8 s, sys: 3.65 s, total: 20.5 s
Wall time: 12min 2s


# v46.3: Run Tests on Punic Cline

In [ ]:
target = "Spain_Punic_oAfrican2"  # The Sicilian "inbred individual"

#sources = ["Anatolia_N", "WHG", "Steppe_EMBA", "Israel_MLBA_Canaanite", "Morocco_EN"]
#sources = ["Morocco_LN.SG", "Morocco_Iberomaurusian", "I12433", "Nigeria_Medieval"]
sources = ["Anatolia_N", "WHG", "Steppe_EMBA"]
sources1 = ["Tunisia_N", "Algeria_IA", "Morocco_LN.SG", "Morocco_EN.SG",
           "Spain_BellBeaker_oAfrican", "Italy_Sicily_MBA_oAfrican", 
            "Italy_Sardinia_C_oAfrican_new", "Spain_EBA_Africa_all"] 

for s1 in sources1:
    leftpops = [target] + [s1] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + ["Morocco_Iberomaurusian"], 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic.v46.3",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "", 
              output_folder = "./output/qpAdm/v46.3/afr_source/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/git/AdmixTools/bin/qpAdm",
              all_snps=True)

In [10]:
print("Finished run!")

Finished run!


### Run several Individuals

# Run distal Models

In [ ]:
### Load the individual IIDs
path_ind = "./eigenstrat/combined/punic1.v43.ind"
outfolder="./output/qpAdm/5wayAll/"

pops = ["Iberia_Tartessian", "Iberia_Iberian", "Iberia_Celtiberian", "Nuragic", 
        "Sicily_Phoenician", "Sicily_IA_Polizzello"]
pops = ["Sicani"]
all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)
all_iids = ["I4246"]

pops_tar = []
pops_ib = []
pops_clt =  []

sources = ["Anatolia_N", "WHG", "Steppe_EMBA",
           "Iran_N", "Morocco_EN"]
add_out = []

print(f"Will run {len(all_iids)} Individuals")
for iid in all_iids[:]:
    print(f"Running Individual {iid}")
    leftpops = [iid] + sources
    qpAdm_run(leftpops = leftpops, 
          rightpops = a12 + add_out, 
          output_file =  str(iid) + ".5way.log", 
          input_folder = "./eigenstrat/combined/", 
          input_file = "punic1.v43",
          par_file_folder = "./parfiles/", 
          input_ind_suff = "_mod_ib", 
          output_folder = outfolder, 
          path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
          all_snps = True)

## Run models with joined output name

In [8]:
### Load the individual IIDs
path_ind = "./eigenstrat/combined/punic1.v43.ind"

pops = ["Sardinia", "Algeria_N", "Punic"]
all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)

all_iids = ["I18195", "MSR002", "I4798", "I12666"]

sources = ["Anatolia_N", "WHG", "Steppe_EMBA", 
           "I12433", "Israel_MLBA_Canaanite"]
add_out = ["Iran_N"]

for iid in all_iids:
    leftpops = [iid] + sources
    qpAdm_run(leftpops = leftpops, 
          rightpops = a12 + add_out, 
          output_file =  ".".join(leftpops) + ".log", 
          input_folder = "./eigenstrat/combined/", 
          input_file = "punic1.v43",
          par_file_folder = "./parfiles/", 
          input_ind_suff = "_mod_ib", 
          output_folder = "./output/qpAdm/5way_Canaanite/", 
          path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
          all_snps = True)

Runtime: 52.080487
Runtime: 52.311518
Runtime: 52.035265
Runtime: 52.738554


## Run 2/3-way Models for African - European Admixture

In [6]:
### Load the individual IIDs
pops = ["Iberia_North_BA_Africa_all", "Algeria_N", 
        "Morocco_LN", "Punic_oAfrican", "Sardinia"]

path_ind = "./eigenstrat/combined/punic1.v43.ind"

all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)
#all_iids = ["I4246"] #The Iberian Outlier

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Anatolia_N", "WHG", "Morocco_EN"]
    add_out = ["Iran_N"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12+add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic0.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod1", 
              output_folder = "./output/qpAdm/3wayNAf/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

Doing Individual: MSR002
Runtime: 32.329889
Doing Individual: MSR003
Runtime: 32.197911
Doing Individual: ORC002
Runtime: 32.199739
Doing Individual: VIL004
Runtime: 32.198779
Doing Individual: VIL006
Runtime: 32.501195
Doing Individual: VIL007
Runtime: 32.166153
Doing Individual: VIL009
Runtime: 32.191764
Doing Individual: VIL010
Runtime: 32.105941
Doing Individual: VIL011
Runtime: 32.232729


# Run 4-way distal models
- with Israel_MLBA_Canaaninte as Outgroup

Add as proxy for North African ancestry:

Morocco_EN, Algeria_N (I12433) or Iberian Outlier (I4246)

In [ ]:
### Load the individual IIDs
pops = ["Italy_Sardinia_Punic", "Italy_Sardinia_Punic_Roman", "Italy_Sardinia_BA_Nuragic",
        "Spain_IA", "Ibiza_Punic.SG", "Italy_Phoenician_Sicily", "Italy_Sicily_Punic",
        "Italy_Sicily_IA_Polizzello"] # "Greece_BA_Mycenaean"

pops1 = ["Villaricos", "Menorca"] # "Himera",
iids = ["VIL", "MSR", "I12517|I15434|I8135"]

#pops = ["Morocco_LN"]
#df1 = get_sub_pops(df, pop_list=pops)
df1 = get_sub_pops_exact(df, pop_list=pops)
#df2 = get_sub_pops(df, pops1, pop_col="loc")
#df3 = get_sub_iid(df, iids)

df_all = pd.concat((df1,))
all_iids = df_all["iid"].values
print(f"Will run {len(all_iids)} Indvidiuals")
# df_meno = get_sub_pops(df, ["Menorca"])
# "Italy_Sardinia_EBA"
# df_cad = get_sub_iid(df, ["I12517|I15434|I8135"])
#all_iids = ["I22112"]

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Anatolia_N", "WHG", "Steppe_EMBA", "I12433"]
    add_out = ["Israel_MLBA_Canaanite"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic.v44",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_ind",   # _mod1
              output_folder = "./output/qpAdm/v44/4way_r_canaanite/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

In [27]:
df[df["clst"].str.contains("Egy")]

NameError: name 'df' is not defined

# Run 4-way Source Models including Myc 
Add as proxy for North African ancestry:
Algeria_N (I12433)

In [ ]:
### Load the individual IIDs
pops = ["Iberia_Punic", "Sardinia", "Iberia_North_BA_Africa_all",
        "Morocco_LN", "Sicily_Phoenician", "Ibiza"]

path_ind = "./eigenstrat/combined/punic1.v43.ind"

all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)
#all_iids = ["I7253"] # Do a single Individual

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Anatolia_N", "WHG", "Steppe_EMBA", "I12433"]
    add_out = ["Israel_MLBA_Canaanite"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic1.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod1", 
              output_folder = "./output/qpAdm/4way_p_Canaanite/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

# Test Continutity to Mycenean Phoenician Group

In [12]:
### Load the individual IIDs

#all_iids = ["I18199", "I18201", "I18203", "I18202", "I18189", "I18187"]
#all_iids = ["MSR002", "MSR003"]
all_iids = ["MS10614.SG"]

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Mycenaean"]
    add_out = ["Israel_MLBA_Canaanite", "I12433"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic1.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod1", 
              output_folder = "./output/qpAdm/t774_cont/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

Doing Individual: MS10614.SG
Runtime: 33.257306


# Run three-way models for Empuries individuals

In [ ]:
pops = ["Iberia_Greek", "Iberia_Hellenistic"]

path_ind = "./eigenstrat/combined/punic1.v43.ind"

all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)

print(f"Will run {len(all_iids)} Individuals")

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Iberia_BA", "Greece_BA_Mycenaean", "I12433"]
    add_out = ["Israel_MLBA_Canaanite"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic1.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod1", 
              output_folder = "./output/qpAdm/Empuries_3way/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

In [24]:
print("test")

test


# Do 3-way distal models for Iberan Individuals (EEF, Steppe, WHG)

In [ ]:
pops = ["Iberia_BA", "Iberia_IA", "Iberia_Tartessian", 
        "Iberia_Iberian", "Iberia_Celtiberian"]
path_ind = "./eigenstrat/combined/punic1.v43.ind"


all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)
#all_iids = ["I18187"] # Do Villaricos Local individual
all_iids = ["I8339", "I18203", "I8206", "I8207"] # Empuries Individuals

print(f"Will run {len(all_iids)} Individuals")

for iid in all_iids[:]:
    print(f"Doing Individual: {iid}")
    sources = ["Anatolia_N", "WHG", "Steppe_EMBA"]
    add_out = []
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic1.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod_ib", 
              output_folder = "./output/qpAdm/3wayIberia/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

# Explore the Ind File (semi Area 51)

In [ ]:
df_ind = pd.read_csv("./eigenstrat/combined/punic1.v43_mod_ib.ind", delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex", "clst"]

In [13]:
df_ind["clst"].value_counts()

Europe_LNBA                              78
Iberia_BA                                42
Europe_EN                                29
Europe_MNChL                             28
Israel_MLBA_Canaanite                    26
                                         ..
Iran_IA                                   1
Israel_MLBA_Canaanite_o_brother.I2200     1
Israel_IA_IIA_Tevet                       1
Crete_Armenoi                             1
Villabruna                                1
Name: clst, Length: 132, dtype: int64

In [14]:
df_ind[df_ind["clst"].str.contains("Polizello")]['clst'].value_counts()

Series([], Name: clst, dtype: int64)

In [28]:
df_ind = pd.read_csv("./eigenstrat/combined/punic.v46.3_punic_clst.ind", delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex","clst"]

In [29]:
df_ind[df_ind["clst"].str.contains("Eg")]

,iid,sex,clst
74,I4526,F,Egypt_Hellenistic
78,I4502,M,Egypt_Roman_oGreek
89,I3339_all,M,Egypt_Hellenistic_Roman_all
90,I3438_all,F,Egypt_Hellenistic_all
91,I3440_all,M,Egypt_Roman_all
99,I4270_all,M,Egypt_Dynastic_all
104,I5958,M,Egypt_Roman
105,I4281_all,F,Egypt_Dynastic_Nubian_all
118,I3438,F,Egypt_Hellenistic
120,I4270,M,Egypt_Dynastic


# Area 51